In [1]:
import pandas as pd
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp
from tqdm import tqdm

In [2]:
df = pd.read_csv('./data_merge.csv')

In [3]:
schema = Object(
    id="meta_data",
    description="Tender meta data",
    attributes=[
        Text(
          id='release_date',
          description="Tender release date",
          examples=[("2023-06-27", "2023-04-11")],
        ),
        Text(
          id='CGN',
          description="CGN of the tender",
          examples=[("CGN-202304240017", "CGN-20180529004")],
        ),
        Text(
            id="brief_description",
            description="Brief description of the tender task",
            examples=[("""Дороги, платформы для подвески, фундаментальные работы в ветряных машинах
                            Результаты в трех пунктах""", 
                       "Установлены результаты первого из трех башен в морской ветряной трубе на острове Янцзы")],
        ),
        Text(
            id="detailed_description",
            description="Detailed description of the tender task",
            examples=[("""Тюрьма Си, Хинпхендо, чайный фонтан и электрическая дорога в рамках проекта Ветер , платформа для подвески и три этапа базовой инженерной работы ветроустановок (прибор)""", 
                       "Первый три башни проекта морской ветровой энергетики на Кананге, Китай, и Яньцзян")],
        ),
        Text(
            id="executor_type",
            description="Type of tenderer company",
            examples=[("Производитель", "Победитель")],
        ),
        Text(
            id="executor_name",
            description="Name of the company executing the tender",
            examples=[("China 19 Metals Group", "Hongong Kong")],
        ),
        Text(
            id="price_currency",
            description="Currency of the tender price",
            examples=[("¥","RMB")]
        ),
        Text(
            id="customer_name",
            description="Name of company customer of the tender",
            examples=[("Ветроэнергетика, Чайна", "Чайна инжиниринг лимитед")],
        ),
        Text(
            id="customer_type",
            description="Type of tender customer company",
            examples=[("Участники торгов", "Заявители")],
        ),  
        Text(
            id="bidding_agent",
            description="Bidding Agent",
            examples=[("Чайна инжиниринг лимитед", "Чайна инжиниринг лимитед")],
        ), 
    ],
    examples=[
        (
            """
            Дата выхода: 2023-06-27
            Дороги, платформы для подвески, фундаментальные работы в ветряных машинах
            Результаты в трех пунктах
            (Контингент: CGN-202304240017)
            Тюрьма Си, Хинпхендо, чайный фонтан и электрическая дорога в рамках проекта &lt; &lt; Ветер &gt; &gt; , платформа для подвески и три этапа базовой инженерной работы ветроустановок (прибор)
            Номер проекта: CGN-202304240017, идентифицированные лица:
            1. Информация о кандидатах:
            Производитель: China 19 Metals Group
            Цены: ¥33 750 545.00
            II. ДРУГИЕ ОБЯЗАТЕЛЬСТВА:
            Нет
            III. УПРАВЛЕНИЕ ВЕРХОВНОГО КОМИССАРА
            Надзор за осуществлением данного тендерного проекта /.
            IV. Связи
            Участники торгов: &lt; &lt; Ветроэнергетика &gt; &gt; , &lt; &lt; Чайна &gt; &gt;
            Контактное лицо: джентльмен
            Тел.: 010-63711966
            Электронная почта: guobijun@cgnpc.com
            Агент по торгам: &quot; Чайна инжиниринг лимитед &quot;
            Контактное лицо: Ли Хун
            Тел.: 0755-84436511
            Электронная почта:
            lhyts@cgnpc.com.cn
            Участник торгов или его главный руководитель (руководитель проекта): (подпись)
            Участник торгов или его агент: (глава)
            """,
            [
                {'release_date':"2023-06-27", 
                 'CGN':"CGN-202304240017", 
                 "brief_description":"""Дороги, платформы для подвески, фундаментальные работы в ветряных машинах
Результаты в трех пунктах""", 
                 "detailed_description":"""Тюрьма Си, Хинпхендо, чайный фонтан и электрическая дорога в рамках проекта Ветер, платформа для подвески и три этапа базовой инженерной работы ветроустановок (прибор)""", 
                 "executor_type":"Производитель", 
                 "executor_name":"China 19 Metals Group", 
                 "price_currency":"¥",
                 "customer_name":"Ветроэнергетика, Чайна", 
                 "customer_type":"Участники торгов", 
                 "bidding_agent":"Чайна инжиниринг лимитед"}
            ],
        ),
        (
            """
            Дата выхода: 2023-04-11
            Установлены результаты первого из трех башен в морской ветряной трубе на острове Янцзы
            (Контингент: CGN-20180529004)
            Первый три башни проекта морской ветровой энергетики на Кананге, Китай, и Яньцзян (заявление No CGN-20180529004)
            Установлено следующее:
            1. Информация о кандидатах:
            Hongong Kong
            Победитель:
            Цены: RMB14 068 380.00
            II. ДРУГИЕ ОБЯЗАТЕЛЬСТВА:
            III. УПРАВЛЕНИЕ ВЕРХОВНОГО КОМИССАРА
            .
            Наблюдение за осуществлением данного тендерного проекта /
            IV. Связи
            Заявители: &quot; Чайна инжиниринг лимитед &quot;
            Местонахождение:/
            Организация: двойная
            Телефон: 0755-88616541
            Электронная почта: luozhenggang@cgnpc.com.cn
            Агент по торгам: &quot; Чайна инжиниринг лимитед &quot;
            Местонахождение:/
            Организация: Худо
            Телефон: 0755-88610728
            Электронная почта:huduo@cgnpc.com.cn
            Участник торгов или его главный руководитель (руководитель проекта): (подпись)
            Участник торгов или его агент: (глава)
            """,
            [
                {'release_date':"2023-04-11", 
                 'CGN':"CGN-20180529004", 
                 "brief_description":"""Установлены результаты первого из трех башен в морской ветряной трубе на острове Янцзы""", 
                 "detailed_description":"""Первый три башни проекта морской ветровой энергетики на Кананге, Китай, и Яньцзян""", 
                 "executor_type":"Победитель", 
                 "executor_name":"Hongong Kong", 
                 "price_currency":"RMB",
                 "customer_name":"Чайна инжиниринг лимитед", 
                 "customer_type":"Заявители", 
                 "bidding_agent":"Чайна инжиниринг лимитед"}
            ],
        ),
    ],
    many=True,
)

In [4]:
llm = LlamaCpp(
    model_path="../models/Mixtral-8x7B-Instruct-v0.1-GGUF/mixtral-8x7b-instruct-v0.1.Q6_K.gguf",
    n_gpu_layers=50,
    n_batch=512,
    n_ctx=3500,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
    temperature = 0
)

In [5]:
chain = create_extraction_chain(llm, schema,encoder_or_encoder_class='json')

In [9]:
result = []
for i_text in tqdm(df['meta']):
    ans = chain.run(text=(i_text))
    result.append(ans['data'])

100%|██████████| 608/608 [2:19:57<00:00, 13.81s/it]  


In [10]:
result

[{'meta_data': [{'release_date': '2023-08-02',
    'CGN': 'CGN-202305310008',
    'brief_description': 'В 2023 году в Китае было приобретено слияние оборудования для смены коробок (первый пункт: винты)',
    'detailed_description': 'Совместимая закупка оборудования для смены коробок в 2023 году (первый пункт: Хиннин)',
    'executor_type': 'Владелец',
    'executor_name': 'SMS',
    'price_currency': 'CNY',
    'customer_name': 'Гуаньчжоу',
    'customer_type': 'Участники торгов',
    'bidding_agent': 'Чайна инжиниринг лимитед'}]},
 {'meta_data': [{'release_date': '2023-01-16',
    'CGN': 'CGN-202211140006',
    'brief_description': 'Закупка оборудования для фотоаппаратов УНИТА (первый пункт: C, D)',
    'detailed_description': 'Закупка оборудования для фотоаппаратов УНИТА (первый пункт: C, D)',
    'executor_type': 'Разработчик',
    'executor_name': 'East Electric Group Eastern Electric Company',
    'price_currency': '¥',
    'customer_name': 'Синьхуа, Коул',
    'customer_type': 'У

In [13]:
temp_list = []
for i in tqdm(range(len(result))):
    if len(result[i]) > 0:
        temp_list.append(result[i]['meta_data'][0])
    else:
     temp_list.append({'release_date':"", 
                 'CGN':"", 
                 "brief_description":"", 
                 "detailed_description":"", 
                 "executor_type":"", 
                 "executor_name":"", 
                 "price_currency":"",
                 "customer_name":"", 
                 "customer_type":"", 
                 "bidding_agent":""})

100%|██████████| 608/608 [00:00<00:00, 1539937.70it/s]


In [14]:
temp_df = pd.DataFrame(temp_list)

In [15]:
temp_df.to_csv('./data/meta_for_merge.csv', index=False)

In [16]:
temp_df

,release_date,CGN,brief_description,detailed_description,executor_type,executor_name,price_currency,customer_name,customer_type,bidding_agent
0,2023-08-02,CGN-202305310008,В 2023 году в Китае было приобретено слияние о...,Совместимая закупка оборудования для смены кор...,Владелец,SMS,CNY,Гуаньчжоу,Участники торгов,Чайна инжиниринг лимитед
1,2023-01-16,CGN-202211140006,Закупка оборудования для фотоаппаратов УНИТА (...,Закупка оборудования для фотоаппаратов УНИТА (...,Разработчик,East Electric Group Eastern Electric Company,¥,"Синьхуа, Коул",Участники торгов,Чайна инжиниринг лимитед
2,2023-08-24,CGN-202307050014,В пункте 1 проекта генерального подряда на про...,1-й сегмент генерального подряда проекта PC на...,Разработчик,China Electric Industrial Co. Ltd.,CNY,China Solar Successor,Участник,Чайна инжиниринг лимитед
3,2023-03-30,CGN-202301310002,Озеро Хэбэй ведает о результатах строительства...,"Вэнь Вэй, Северное озеро, попыталось закупить ...",Заявители,Китайская гидроэнергетическая компания 12-я ин...,CNY,China's New Energy Corp.,Участник,Чайна инжиниринг лимитед
4,2023-09-13,CGN-202307040015,2023 - 2024 гг. в матрице фотоэлектрических ко...,С 2023 по 2024 гг. рамка фотоэлектрических ком...,Разработчик,Lonk Green Energy,CNY,Чайна энтропия,Приглашение,Чайна инжиниринг лимитед
...,...,...,...,...,...,...,...,...,...,...
603,2023-04-13,CGN-20221220004,Объявлены результаты закупки оборудования для ...,Закупка ветрогенераторного оборудования для пр...,Инвесторы,Smart Energy Group,CNY,"Южно-Ветроэнергетическая компания, Гонконг, Китай",Участники торгов,Чайна инжиниринг лимитед
604,2023-03-13,CGN-202301180008,Конкретная информация о результатах в рамках о...,Проект по закупке материалов для комплексной м...,Партнеры,Нанкинская корпорация по техническому обслужив...,CNY,Хинси Порт оф энтерпрайз,Заявители,Чайна инжиниринг лимитед
605,2023-07-31,CGN-202305310011,В 2023 году в Китае было приобретено третье об...,Совместимая закупка третьего коробочного обору...,Производитель,China Electric Company,CNY,China and China New Energy Co. Ltd.,Участник,Чайна инжиниринг лимитед
606,2023-05-06,CGN-20230103001,"Проект 1 и 2 группы LOT88Ea, блок управления д...",LOT88Ea для клапанов по борьбе с курением и ко...,Продюсер,Гуандун,CNY,Чайна инжиниринг лимитед,Заявители,Чайна инжиниринг лимитед
